In [4]:

pip install psycopg2


Note: you may need to restart the kernel to use updated packages.


In [98]:
import psycopg2

In [99]:
#import necessary dependencies
import pandas as pd
from datetime import date

In [100]:
#connect to existing database postgres
def get_db_connection():
    connection=psycopg2.connect(
        host='localhost',
        database='postgres',
        user='postgres',
        password='Jimbiya1!',
        port=5433
    )
    return connection

conn=get_db_connection()

In [25]:

#to create table retail order
def create_tables():
    conn = get_db_connection()
    cursor = conn.cursor()
    create_table_query = '''

                        drop table if exists retail_orders cascade;
                        
                        create table if not exists retail_orders(
                        Row_ID integer,
                        Order_ID varchar(1000),
                        Order_Date varchar(1000),
                        Ship_Date varchar(1000),
                        Ship_Mode varchar(1000),
                        Customer_ID varchar(1000),
                        Customer_Name varchar(1000),
                        Segment varchar(1000),
                        Country varchar(1000),
                        City varchar(1000),
                        State varchar(1000),
                        Postal_Code float,
                        Region varchar(1000),
                        Product_ID varchar(1000),
                        Category varchar(1000),
                        Sub_Category varchar(1000),
                        Product_Name varchar(1000),
                        Sales float
                        );'''
    cursor.execute(create_table_query)
    conn.commit()
    cursor.close()
    conn.close()

create_tables()

In [7]:
#Reading the dataset into dataFrames
haven_retail = pd.read_csv('Haven_dataset/Haven Sales Dataset.csv')

In [8]:
haven_retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9800 non-null   int64  
 1   Order ID       9800 non-null   object 
 2   Order Date     9800 non-null   object 
 3   Ship Date      9800 non-null   object 
 4   Ship Mode      9800 non-null   object 
 5   Customer ID    9800 non-null   object 
 6   Customer Name  9800 non-null   object 
 7   Segment        9800 non-null   object 
 8   Country        9800 non-null   object 
 9   City           9800 non-null   object 
 10  State          9800 non-null   object 
 11  Postal Code    9789 non-null   float64
 12  Region         9800 non-null   object 
 13  Product ID     9800 non-null   object 
 14  Category       9800 non-null   object 
 15  Sub-Category   9800 non-null   object 
 16  Product Name   9800 non-null   object 
 17  Sales          9800 non-null   float64
dtypes: float

In [35]:
haven_retail.fillna({
'Postal Code': 0.00
},inplace=True)

In [36]:
haven_retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9800 non-null   int64  
 1   Order ID       9800 non-null   object 
 2   Order Date     9800 non-null   object 
 3   Ship Date      9800 non-null   object 
 4   Ship Mode      9800 non-null   object 
 5   Customer ID    9800 non-null   object 
 6   Customer Name  9800 non-null   object 
 7   Segment        9800 non-null   object 
 8   Country        9800 non-null   object 
 9   City           9800 non-null   object 
 10  State          9800 non-null   object 
 11  Postal Code    9800 non-null   float64
 12  Region         9800 non-null   object 
 13  Product ID     9800 non-null   object 
 14  Category       9800 non-null   object 
 15  Sub-Category   9800 non-null   object 
 16  Product Name   9800 non-null   object 
 17  Sales          9800 non-null   float64
dtypes: float

In [38]:
#Save clean data to csv
haven_retail.to_csv('Haven_dataset/clean_retail_orders.csv',index=False)

In [39]:
#2 to load data from csv to into retail table
#function to load data into table
import csv
def load_data_from_csv2(csv_path):
    conn = get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r') as file:
        reader= csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute(
                ''' insert into retail_orders (Row_ID, Order_ID, Order_Date, Ship_Date, Ship_Mode, Customer_ID, Customer_Name, 
                Segment, Country, City, State, Postal_Code, Region, Product_ID, Category, Sub_Category, Product_Name, Sales) 
                    values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s );''',
                       row
                )
    conn.commit()
    cursor.close()
    conn.close()

#provide the csv path to the file
csv_file_path = r'Haven_dataset/clean_retail_orders.csv'

load_data_from_csv2(csv_file_path)
print('retail_orders data loaded successfully')

retail_orders data loaded successfully


In [59]:
#3. code to check if retail_orders table exist after creation
conn = get_db_connection()
cursor = conn.cursor()
cursor.execute('''
SELECT EXISTS ( 
	SELECT 1
	FROM pg_tables
	WHERE tablename = 'retail_orders')''')
exists = cursor.fetchone()
exists

(True,)

In [93]:
#4. code to fetch and print the first five rows of data from table

conn = get_db_connection()
cursor = conn.cursor()
cursor.execute('''
		SELECT *
		FROM retail_orders
    	limit 5''')
print([desc[0] for desc in cursor.description])
cursor.fetchall()

['row_id', 'order_id', 'order_date', 'ship_date', 'ship_mode', 'customer_id', 'customer_name', 'segment', 'country', 'city', 'state', 'postal_code', 'region', 'product_id', 'category', 'sub_category', 'product_name', 'sales']


[(9800,
  'CA-2016-128608',
  '12-01-16',
  '17-01-16',
  'Standard Class',
  'CS-12490',
  'Cindy Schnelling',
  'Corporate',
  'United State',
  'Toledo',
  'Ohio',
  43615.0,
  'East',
  'TEC-AC-10000487',
  'Technology',
  'Accessories',
  'SanDisk Cruzer 4GB USB Flash Drive',
  10.384),
 (1,
  'CA-2017-152156',
  '08-11-17',
  '11-11-17',
  'Second Class',
  'CG-12520',
  'Claire Gute',
  'Consumer',
  'United States',
  'Henderson',
  'Kentucky',
  42420.0,
  'South',
  'FUR-BO-10001798',
  'Furniture',
  'Bookcases',
  'Bush Somerset Collection Bookcase',
  261.96),
 (2,
  'CA-2017-152156',
  '08-11-17',
  '11-11-17',
  'Second Class',
  'CG-12520',
  'Claire Gute',
  'Consumer',
  'United States',
  'Henderson',
  'Kentucky',
  42420.0,
  'South',
  'FUR-CH-10000454',
  'Furniture',
  'Chairs',
  'Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back',
  731.94),
 (3,
  'CA-2017-138688',
  '12-06-17',
  '16-06-17',
  'Second Class',
  'DV-13045',
  'Darrin Van Huff',
  'C

In [77]:
#5. write SQL query to retrieve the total number of orders

conn = get_db_connection()
cursor = conn.cursor()
cursor.execute('''
		SELECT count(order_id) as total_orders
		FROM retail_orders ''')
print([desc[0] for desc in cursor.description])
cursor.fetchall()

['total_orders']


[(9800,)]

In [78]:
#6. write SQL query to calculate the total sales amount for all orders

conn = get_db_connection()
cursor = conn.cursor()
cursor.execute('''
		SELECT sum(sales) as total_sales
		FROM retail_orders ''')
print([desc[0] for desc in cursor.description])
cursor.fetchall()

['total_sales']


[(2261536.7826999533,)]

In [96]:
#7. python function to fetch all orders placed in Florida

def fetchorder():
    try:
        # Open the connection
        conn = get_db_connection()
        cursor = conn.cursor()
        cursor.execute("""select * FROM retail_orders where state = 'Florida'""")
        print([desc[0] for desc in cursor.description])
        rows=cursor.fetchall()
        print("The number of orders placed in Florida:", cursor.rowcount)
        for row in rows:
            print(row)
        #close the connection
        conn.commit()
        cursor.close()
        conn.close()
    except Exception as e:
        print("An error occurred:")
fetchorder()

['row_id', 'order_id', 'order_date', 'ship_date', 'ship_mode', 'customer_id', 'customer_name', 'segment', 'country', 'city', 'state', 'postal_code', 'region', 'product_id', 'category', 'sub_category', 'product_name', 'sales']
The number of orders placed in Florida: 373
(4, 'US-2016-108966', '11-10-16', '18-10-16', 'Standard Class', 'SO-20335', "Sean O'Donnell", 'Consumer', 'United States', 'Fort Lauderdale', 'Florida', 33311.0, 'South', 'FUR-TA-10000577', 'Furniture', 'Tables', 'Bretford CR4500 Series Slim Rectangular Table', 957.5775)
(5, 'US-2016-108966', '11-10-16', '18-10-16', 'Standard Class', 'SO-20335', "Sean O'Donnell", 'Consumer', 'United States', 'Fort Lauderdale', 'Florida', 33311.0, 'South', 'OFF-ST-10000760', 'Office Supplies', 'Storage', "Eldon Fold 'N Roll Cart System", 22.368)
(44, 'CA-2018-139619', '19-09-18', '23-09-18', 'Standard Class', 'ES-14080', 'Erin Smith', 'Corporate', 'United States', 'Melbourne', 'Florida', 32935.0, 'South', 'OFF-ST-10003282', 'Office Suppli

In [117]:
#8. SQL query to find the top 5 customers based on their total sales
conn = get_db_connection()
cursor = conn.cursor()
cursor.execute("""SELECT customer_name, SUM(sales) as total_sales 
FROM retail_orders 
GROUP BY customer_name 
ORDER BY total_sales DESC LIMIT 5""")
print([desc[0] for desc in cursor.description])
cursor.fetchall()

['customer_name', 'total_sales']


[('Sean Miller', 25043.05),
 ('Tamara Chand', 19052.217999999997),
 ('Raymond Buch', 15117.339),
 ('Tom Ashbrook', 14595.62),
 ('Adrian Barton', 14473.570999999998)]

In [82]:
#9. python script to insert new order into the table
conn = get_db_connection()
cursor = conn.cursor()
m = ("""INSERT into retail_orders (Row_ID, Order_ID, Order_Date, Ship_Date, Ship_Mode, Customer_ID, Customer_Name, 
                Segment, Country, City, State, Postal_Code, Region, Product_ID, Category, Sub_Category, Product_Name, Sales) 
                    values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s );
                """)
val = [9800, 'CA-2016-128608', '12-01-16', '17-01-16', 'Standard Class','CS-12490','Cindy Schnelling', 'Corporate', 'United State', 'Toledo', 'Ohio', 43615, 'East', 'TEC-AC-10000487','Technology','Accessories', 'SanDisk Cruzer 4GB USB Flash Drive', 10.384]
cursor.execute(m, val)
conn.commit()

In [85]:
# 10. write SQL query to count number of orders using Standard Class

conn = get_db_connection()
cursor = conn.cursor()
cursor.execute("""
		SELECT count(order_id) as Standard_Class_orders
		FROM retail_orders  WHERE ship_mode = 'Standard Class'""")
print([desc[0] for desc in cursor.description])
cursor.fetchall()

['standard_class_orders']


[(5859,)]

In [104]:
#11. python function to update the ship mode to standard class

def updateorder():
    try:
        # Open the connection
        conn = get_db_connection()
        cursor = conn.cursor()
        cursor.execute("""UPDATE retail_orders SET ship_mode = 'Standard Class' where order_id = 'CA-2015-167164'""")
        print("Updated successfully")
        #close the connection
        conn.commit()
        cursor.close()
        conn.close()
    except Exception as e:
        print(f"An error occurred: {e}")
updateorder()

Updated successfully


In [ ]:
#12. write a python script to delete order



In [118]:
#13. python function to calc total sales 
def totalsales():
    try:
        # Open the connection
        conn = get_db_connection()
        cursor = conn.cursor()
        cursor.execute("""SELECT SUM(sales) from retail_orders 
                       where category = 'Technology'""")
        print([desc[0] for desc in cursor.description])
        cursor.fetchall()
       
        #close the connection
        #conn.commit()
        #cursor.close()
        #conn.close()
    except Exception as e:
        print(f"An error occurred: {e}")
totalsales()

['sum']
